# 0 - Imports

In [13]:
# os: Proporciona funciones para interactuar con el sistema operativo, como manejar rutas de archivos y directorios.
import os

# time: Permite acceder a funciones que manejan el tiempo, como pausas (sleep) o tomar el tiempo actual.
import time

# uuid: Usado para generar identificadores únicos universales (UUIDs), que son útiles para nombres de archivos únicos, por ejemplo.
import uuid

# cv2: OpenCV para operaciones relacionadas con visión por computadora, como procesamiento de imágenes y video.
import cv2

# tensorflow as tf: Biblioteca para crear modelos de aprendizaje automático, especialmente redes neuronales profundas.
import tensorflow as tf

# json: Biblioteca para trabajar con datos en formato JSON, útil para serializar y deserializar datos.
import json

# numpy as np: Proporciona soporte para arrays y matrices grandes y funciones matemáticas de alto nivel para operar con estos.
import numpy as np

# matplotlib.pyplot as plt: Proporciona una interfaz para crear gráficos y visualizaciones estáticas, animadas e interactivas en Python.
from matplotlib import pyplot as plt

# albumentations as alb: Biblioteca de aumento de imágenes que proporciona diversas técnicas de transformación de imágenes para mejorar los conjuntos de datos de entrenamiento de visión por computadora.
import albumentations as alb


# 1 - Colación de imagenes

Para poder entrenar la red neuronal necesitamos un conjunto de imágenes con los puntos iris etiquetatdos.

Para ello la idea que se tiene es realizar un video y que de dicho video se selleciónene 80 imagenes que luego mediante la aplicación de **labelme** se etiquetaran los puntos (ojo derecho y izquierdo), para obtener sus cordenadas

In [6]:
IMAGES_PATH = 'data'
number_images = 80

In [15]:
# Iniciar la captura de video desde la cámara predeterminada (cámara 0)
cap = cv2.VideoCapture(0)

# Bucle para capturar un número específico de imágenes
for imgnum in range(number_images):
    # Imprimir en consola el número de la imagen que se está capturando
    print('Collecting image {}'.format(imgnum))

    # Capturar un frame de la cámara
    ret, frame = cap.read()

    # Generar un nombre único para la imagen utilizando uuid y guardar la imagen
    imgname = os.path.join(IMAGES_PATH, f'{str(uuid.uuid1())}.jpg')
    cv2.imwrite(imgname, frame)

    # Mostrar el frame capturado en una ventana
    cv2.imshow('frame', frame)

    # Esperar medio segundo antes de capturar la siguiente imagen
    time.sleep(0.5)

    # Comprobar si se presionó la tecla 'q' para salir del bucle
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la cámara y cerrar todas las ventanas de OpenCV
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.6.0) C:\b\abs_f8n1j3l9l0\croot\opencv-suite_1691622637237\work\modules\highgui\src\window.cpp:1267: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


# 2 - Crear el dataset

In [ ]:
def load_image(x):
    # Leer el archivo de imagen desde la ruta proporcionada y almacenarlo como un conjunto de bytes.
    # 'x' es la ruta de la imagen que se va a cargar.
    byte_img = tf.io.read_file(x)

    # Decodificar la imagen en formato JPEG de los bytes leídos.
    # Esto convierte los datos de la imagen de su formato de archivo (JPEG) a un tensor que TensorFlow puede utilizar.
    img = tf.io.decode_jpeg(byte_img)

    # Retornar el tensor de la imagen decodificada.
    return img

# 3 - Mover las imágenes y anotaciones

In [26]:
# Iterar a través de las carpetas 'train', 'test', y 'val'
for folder in ['train', 'test', 'val']:
    # Iterar sobre cada archivo en la subcarpeta 'images' dentro de cada carpeta 'train', 'test', 'val'
    for file in os.listdir(os.path.join('data', folder, 'images')):

        # Construir el nombre del archivo JSON correspondiente al archivo de imagen
        # Se asume que el nombre del archivo de imagen y del archivo JSON son iguales excepto la extensión
        filename = file.split('.')[0] + '.json'

        # Construir la ruta completa del archivo JSON en la carpeta 'labels'
        existing_filepath = os.path.join('data', 'labels', filename)

        # Comprobar si el archivo JSON existe
        if os.path.exists(existing_filepath):
            # Construir la nueva ruta donde se debería mover el archivo JSON
            # Esta ruta coloca el archivo JSON en una subcarpeta 'labels' dentro de 'train', 'test' o 'val'
            new_filepath = os.path.join('data', folder, 'labels', filename)

            # Mover (reemplazar) el archivo desde su ubicación actual a la nueva ruta
            os.replace(existing_filepath, new_filepath)

# 4 - Modificar las imagenes

In [29]:
# Importamos el módulo 'albumentations' como 'alb'. Este módulo se utiliza para la 
# augmentación de imágenes, es decir, para aplicar transformaciones que generan 
# variaciones de las imágenes existentes.

augmentor = alb.Compose([
    # Aplica un recorte aleatorio en la imagen con un ancho y alto de 450 píxeles.
    alb.RandomCrop(width=450, height=450), 

    # Realiza un volteo horizontal (espejo) de la imagen con una probabilidad del 50%.
    alb.HorizontalFlip(p=0.5), 

    # Ajusta aleatoriamente el brillo y el contraste de la imagen con una probabilidad del 20%.
    alb.RandomBrightnessContrast(p=0.2),

    # Aplica una corrección gamma aleatoria a la imagen con una probabilidad del 20%.
    # Esto puede afectar la iluminación de la imagen.
    alb.RandomGamma(p=0.2), 

    # Cambia aleatoriamente los canales de color RGB de la imagen con una probabilidad del 20%.
    alb.RGBShift(p=0.2), 

    # Realiza un volteo vertical de la imagen con una probabilidad del 50%.
    alb.VerticalFlip(p=0.5)
], 
# Configura los parámetros para las transformaciones que involucran puntos clave.
# 'format' especifica el formato de los puntos clave (en este caso 'xy' para coordenadas cartesianas).
# 'label_fields' especifica los campos de etiquetas que se deben utilizar con los puntos clave.
keypoint_params=alb.KeypointParams(format='xy', label_fields=['class_labels']))


In [30]:
for partition in ['train', 'test', 'val']: 
    # Itera sobre cada imagen en el directorio especificado.
    for image in os.listdir(os.path.join('data', partition, 'images')):
        # Lee la imagen usando OpenCV.
        img = cv2.imread(os.path.join('data', partition, 'images', image))

        # Inicializa clases y coordenadas por defecto.
        classes = [0,0]
        coords = [0,0,0.00001,0.00001]

        # Construye la ruta al archivo de etiquetas correspondiente a la imagen.
        label_path = os.path.join('data', partition, 'labels', f'{image.split(".")[0]}.json')
        
        # Verifica si el archivo de etiquetas existe.
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label = json.load(f)
                
        # Procesa las etiquetas, extrayendo clases y coordenadas de los puntos clave.
            if label['shapes'][0]['label']=='LeftEye': 
                classes[0] = 1
                coords[0] = np.squeeze(label['shapes'][0]['points'])[0]
                coords[1] = np.squeeze(label['shapes'][0]['points'])[1]

            if label['shapes'][0]['label']=='RightEye':
                classes[1] = 1
                coords[2] = np.squeeze(label['shapes'][0]['points'])[0]
                coords[3] = np.squeeze(label['shapes'][0]['points'])[1]

            if len(label['shapes']) > 1:     
                if label['shapes'][1]['label'] =='LeftEye': 
                    classes[0] = 1 
                    coords[0] = np.squeeze(label['shapes'][1]['points'])[0]
                    coords[1] = np.squeeze(label['shapes'][1]['points'])[1]

                if label['shapes'][1]['label'] =='RightEye': 
                    classes[1] = 1
                    coords[2] = np.squeeze(label['shapes'][1]['points'])[0]
                    coords[3] = np.squeeze(label['shapes'][1]['points'])[1]
            
            np.divide(coords, [640,480,640,480])
                
        try: 
            for x in range(120):
                keypoints = [(coords[:2]), (coords[2:])]
                augmented = augmentor(image=img, keypoints=keypoints, class_labels=['LeftEye','RightEye'])
                cv2.imwrite(os.path.join('aug_data', partition, 'images', f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])

                annotation = {}
                annotation['image'] = image
                annotation['class'] = [0,0]
                annotation['keypoints'] = [0,0,0,0]

                if os.path.exists(label_path):
                    if len(augmented['keypoints']) > 0: 
                        for idx, cl in enumerate(augmented['class_labels']):
                            if cl == 'LeftEye': 
                                annotation['class'][0] = 1 
                                annotation['keypoints'][0] = augmented['keypoints'][idx][0]
                                annotation['keypoints'][1] = augmented['keypoints'][idx][1]
                            if cl == 'RightEye': 
                                annotation['class'][1] = 1 
                                annotation['keypoints'][2] = augmented['keypoints'][idx][0]
                                annotation['keypoints'][3] = augmented['keypoints'][idx][1]
                                
                annotation['keypoints'] = list(np.divide(annotation['keypoints'], [450,450,450,450]))


                with open(os.path.join('aug_data', partition, 'labels', f'{image.split(".")[0]}.{x}.json'), 'w') as f:
                    json.dump(annotation, f)

        except Exception as e:
            print(e)